## Overview of Triggers in Spark Structured Streaming

Let us get an overview of triggers supported by Spark Structured Streaming.

The trigger settings of a streaming query define the timing of streaming data processing, whether the query is going to be executed as micro-batch query with a fixed batch interval or as a continuous processing query. Here are the different kinds of triggers that are supported.

Here are the different trigger types:
* Unspecified (Default)
* Fixed interval micro-batches
* One-time micro-batch
* Continuous with fixed checkpoint interval (experimental)

**Unspecified (Default)**

If no trigger setting is explicitly specified, then by default, the query will be executed in micro-batch mode, where micro-batches will be generated as soon as the previous micro-batch has completed processing.

**Fixed interval micro-batches**

The query will be executed with micro-batches mode, where micro-batches will be kicked off at the user-specified intervals.
* If the previous micro-batch completes within the interval, then the engine will wait until the interval is over before kicking off the next micro-batch.
* If the previous micro-batch takes longer than the interval to complete (i.e. if an interval boundary is missed), then the next micro-batch will start as soon as the previous one completes (i.e., it will not wait for the next interval boundary).
* If no new data is available, then no micro-batch will be kicked off.

**One-time micro-batch**

The query will execute **only one** micro-batch to process all the available data and then stop on its own. This is useful in scenarios you want to periodically spin up a cluster, process everything that is available since the last period, and then shutdown the cluster. In some case, this may lead to significant cost savings.

**Continuous with fixed checkpoint interval (experimental)**

The query will be executed in the new low-latency, continuous processing mode. 

Launch Pyspark using below commands and run Spark Structured Streaming Code.

**Using Pyspark2**

```
export PYSPARK_PYTHON=python3

pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark3**

```
export PYSPARK_PYTHON=python3

pyspark3 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

So far we have seen examples using **Fixed interval micro-batches**. We will see other examples in subsequent sections.

```python
spark.conf.set('spark.sql.shuffle.partitions', '2')

import socket
hostname = socket.gethostname()

log_messages = spark. \
    readStream. \
    format("socket"). \
    option("host", hostname). \
    option("port", 9000). \
    load()

log_messages. \
    writeStream. \
    outputMode("append"). \
    format("console"). \
    option('truncate', 'false'). \
    trigger(processingTime='5 seconds'). \
    start()
```